In [ ]:
import os
import json
import math
from collections import Counter
import numpy as np
import networkx as nx

os.chdir("..")
print(os.getcwd())

In [ ]:
def get_path_mapping(data):
    path_dict = {}
    for p in data:
        path_dict[p["path_id"]] = p
    return path_dict


def load_nav_graphs(scans):
    ''' Load connectivity graph for each scan '''

    def distance(pose1, pose2):
        ''' Euclidean distance between two graph poses '''
        return ((pose1['pose'][3]-pose2['pose'][3])**2\
          + (pose1['pose'][7]-pose2['pose'][7])**2\
          + (pose1['pose'][11]-pose2['pose'][11])**2)**0.5

    graphs = {}
    for scan in scans:
        with open('connectivity/%s_connectivity.json' % scan) as f:
            G = nx.Graph()
            positions = {}
            data = json.load(f)
            for i,item in enumerate(data):
                if item['included']:
                    for j,conn in enumerate(item['unobstructed']):
                        if conn and data[j]['included']:
                            positions[item['image_id']] = np.array([item['pose'][3],
                                    item['pose'][7], item['pose'][11]]);
                            assert data[j]['unobstructed'][i], 'Graph should be undirected'
                            G.add_edge(item['image_id'],data[j]['image_id'],weight=distance(item,data[j]))
            nx.set_node_attributes(G, values=positions, name='position')
            graphs[scan] = G
    return graphs

def fix_path(node_path):
    fixed_path = []
    current = None
    for node in node_path:
        if node != current:
            fixed_path.append(node)
            
        current = node
        
    return fixed_path
        
def get_path_length(graph, node_path):
    path_length = 0
    
    for i in range(len(node_path)-1):
        u, v = node_path[i], node_path[i+1]
        path_length += graph[u][v]["weight"]
        
    return path_length

def get_osr(graph, node_path, gold_node, threshold=3.0):
    osr = 0
    for node in node_path:
        shortest_path = nx.shortest_path(graph, source=node, target=gold_node, weight="weight")
        navigation_error = sum(graph[u][v]["weight"] for u, v in zip(shortest_path[:-1], shortest_path[1:]))
        
        if navigation_error < threshold:
            osr = 1
            break
            
    return osr

def calculate_pc(reference, predicted, graph, threshold=3.0):
    pc = 0
    
    for ref in reference:
        min_distance = 1000
        for pred in predicted:
            shortest_path = nx.shortest_path(graph, source=ref, target=pred, weight="weight")
            distance = sum(graph[u][v]["weight"] for u, v in zip(shortest_path[:-1], shortest_path[1:]))
            
            min_distance = min(distance, min_distance)
            
        
        pc += math.exp(-(min_distance/threshold))
        
    return pc/len(reference)

def calculate_epl(reference, predicted, pc, graph):
    epl = pc * reference["distance"]
    return epl

def calculate_ls(reference, predicted, pc, graph):
    epl = calculate_epl(reference, predicted, pc, graph)
    pl = get_path_length(graph, predicted)
    ls = epl/(epl+ abs(epl-pl))
    return ls
    

def get_results(r2r_data, model_path, actions, threshold=3.0):
    with open(model_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    print(len(data))
    r2r_mapping = get_path_mapping(r2r_data)
    scans = set([p["scan"] for p in r2r_mapping.values()])

    G = load_nav_graphs(scans)
    
    sr = 0
    ne = 0
    pl = 0
    osr = 0
    spl = 0
    cls = 0
    
    success_paths = []
    longest_path = 0
    longest_path_id = ""
    
    for path_id, path in data.items():
        path_data = r2r_mapping[int(path_id)]
        gold_node = path_data["path"][-1]
        tmp_g = G[path_data["scan"]]
        old_path = path
        path = fix_path(path)
        path_length = get_path_length(tmp_g, path)
        pl += get_path_length(tmp_g, path)
        
        pc = calculate_pc(path_data["path"], path, tmp_g, threshold=threshold)
        cls += pc * calculate_ls(path_data, path, pc, tmp_g)
        
        # Calculate the shortest path between stop node and goal node
        shortest_path = nx.shortest_path(tmp_g, source=path[-1], target=gold_node, weight="weight")
        navigation_error = sum(tmp_g[u][v]["weight"] for u, v in zip(shortest_path[:-1], shortest_path[1:]))
        
        ne += navigation_error      
        osr += get_osr(tmp_g, path, gold_node, threshold=threshold)
        
        success_indicator = 0
        # navigation error less than 3 = success
        if navigation_error < threshold and len(old_path) < 32 and actions[str(path_id)][-1] == "Stop":
            success_paths.append(path_id)
            sr += 1
            
            success_indicator = 1

            if len(path) > longest_path:
                longest_path = len(path)
                longest_path_id = path_id
                
        spl += (success_indicator*(path_data["distance"]/max(path_data["distance"], path_length)))
        
    success_rate = sr/len(data)  
    navigation_error = ne/len(data)
    average_path_length = pl/len(data)
    oracle_success_rate = osr/len(data)
    spl = spl/len(data)
    cls = cls/len(data)
    
    return success_rate, navigation_error, average_path_length, oracle_success_rate, spl, cls, sr

## Main results

In [ ]:
import json
import pandas as pd

model = "qwen2_5"
split = "unseen"
action_space = "non-panoramic"

# Load R2R data
with open(f"tasks/R2R/data/R2R_val_{split}.json", "r", encoding="utf-8") as file:
    r2r_data = json.load(file)

# Load action data
#actions_files = [
#    f"./model_paths/actions_{model}_{action_space}_{split}_inst_0.json",
#    f"./model_paths/actions_{model}_{action_space}_{split}_inst_1.json",
#    f"./model_paths/actions_{model}_{action_space}_{split}_inst_2.json"
#]

actions_files = [
    f"./model_paths/full-data/low-level/actions_{model}_{action_space}_full_{split}_inst_0.json",
    #f"./model_paths/full-data/low-level/actions_{model}_{action_space}_full_{split}_inst_1.json",
    #f"./model_paths/full-data/low-level/actions_{model}_{action_space}_full_{split}_inst_2.json"
]

actions_data = [json.load(open(f, "r", encoding="utf-8")) for f in actions_files]

# Load model path data
#model_paths = [
#    f"./model_paths/{model}_{action_space}_{split}_inst_0.json",
#    f"./model_paths/{model}_{action_space}_{split}_inst_1.json",
#    f"./model_paths/{model}_{action_space}_{split}_inst_2.json"
#]

model_paths = [
    f"./model_paths/full-data/low-level/{model}_{action_space}_full_{split}_inst_0.json",
    #f"./model_paths/full-data/low-level/{model}_{action_space}_full_{split}_inst_1.json",
    #f"./model_paths/full-data/low-level/{model}_{action_space}_full_{split}_inst_2.json"
]

# Get results for each model path
results = [get_results(r2r_data, model_paths[i], actions_data[i]) for i in range(1)]

In [ ]:
# Create a DataFrame
df = pd.DataFrame({
    "PL": [r[2] for r in results],
    "NE": [r[1] for r in results],
    "OSR": [r[3] for r in results],
    "SR": [r[0] for r in results],
    "SPL": [r[4] for r in results],
    "CLS": [r[5] for r in results],
})

# Compute mean and std correctly
summary = pd.DataFrame({
    "Mean": df.mean(),
    "Std": df.std()
})

# Round results only at the end
summary_rounded = summary.round(2)

print(summary_rounded)
# Compute the average for each column except the index
df.loc["Average"] = df.mean(numeric_only=True).round(20)
df.loc["Std"] = df.std().round(3)
df.iloc[:-1, 1:] = df.iloc[:-1, 1:].applymap(lambda x: f"{x:.13f}")
# Print the DataFrame
print(df)

In [ ]:
model_path1 = "./model_paths/qwen2_5_non_panoramic_seen_inst_0.json"
model_path2 = "./model_paths/qwen2_5_non_panoramic_seen_inst_1.json"
model_path3 = "./model_paths/qwen2_5_non_panoramic_seen_inst_2.json"

# husk å vis SR når NE endrer seg for å se hvor ofte den lander på akkurat den noden den skal.
with open("tasks/R2R/data/R2R_val_seen.json", "r", encoding="utf-8") as file:
    r2r_data = json.load(file)
    
with open("./model_paths/actions_qwen2_5_non_panoramic_seen_inst_0.json", "r", encoding="utf-8") as file:
    actions = json.load(file)
    
with open("./model_paths/actions_qwen2_5_non_panoramic_seen_inst_0.json", "r", encoding="utf-8") as file:
    actions2 = json.load(file)
    
with open("./model_paths/actions_qwen2_5_non_panoramic_seen_inst_0.json", "r", encoding="utf-8") as file:
    actions3 = json.load(file)
    
print(f"Length of val unseen: {len(r2r_data)}")

print(actions2 == actions)

model_path1_data = get_results(r2r_data, model_path1, actions)
model_path2_data = get_results(r2r_data, model_path2, actions2)
model_path3_data = get_results(r2r_data, model_path3, actions3)

model_path1_sr = model_path1_data[0]
model_path2_sr = model_path2_data[0]
model_path3_sr = model_path3_data[0]

model_path1_ne = model_path1_data[1]
model_path2_ne = model_path2_data[1]
model_path3_ne = model_path3_data[1]

model_path1_pl = model_path1_data[2]
model_path2_pl = model_path2_data[2]
model_path3_pl = model_path3_data[2]

model_path1_osr = model_path1_data[3]
model_path2_osr = model_path2_data[3]
model_path3_osr = model_path3_data[3]

model_path1_spl = model_path1_data[4]
model_path2_spl= model_path2_data[4]
model_path3_spl = model_path3_data[4]

model_path1_cls = model_path1_data[5]
model_path2_cls= model_path2_data[5]
model_path3_cls = model_path3_data[5]

avg_sucess_rate = (model_path1_sr+model_path2_sr+model_path3_sr)/3
print(f"Sucess rate index 0: {round(model_path1_sr, 2)}")
print(f"Sucess rate index 1: {round(model_path2_sr, 2)}")
print(f"Sucess rate index 2: {round(model_path3_sr, 2)}")

print(f"Navigation Error index 0: {round(model_path1_ne, 2)}")
print(f"Navigation Error index 1: {round(model_path2_ne, 2)}")
print(f"Navigation Error index 2: {round(model_path3_ne, 2)}")

print(f"Path Length index 0: {round(model_path1_pl, 2)}")
print(f"Path Length index 1: {round(model_path2_pl, 2)}")
print(f"Path length index 2: {round(model_path3_pl, 2)}")

print(f"Oracle Success Rate index 0: {round(model_path1_osr, 2)}")
print(f"Oracle Success Rate index 1: {round(model_path2_osr, 2)}")
print(f"Oracle Success Rate index 2: {round(model_path3_osr, 2)}")

print(f"Success Rate Weighted By Path Length index 0: {round(model_path1_spl, 2)}")
print(f"Success Rate Weighted By Path Length index 1: {round(model_path2_spl, 2)}")
print(f"Success Rate Weighted By Path Length index 2: {round(model_path3_spl, 2)}")

print(f"Coverage Weighted By Length Score index 0: {round(model_path1_cls, 2)}")
print(f"Coverage Weighted By Length Score index 1: {round(model_path2_cls, 2)}")
print(f"Coverage Weighted By Length Score index 2: {round(model_path3_cls, 2)}")

print(f"Average Success Rate for validation unseen data: {round(avg_sucess_rate, 2)}")

In [ ]:
with open("tasks/R2R/data/R2R_val_unseen.json", "r", encoding="utf-8") as file:
    r2r_data = json.load(file)

path = [
      "c9e8dc09263e4d0da77d16de0ecddd39", 
      "f33c718aaf2c41469389a87944442c62", 
      "ae91518ed77047b3bdeeca864cd04029", 
    ]
path_data = r2r_data[0]
G = load_nav_graphs(["8194nk5LbLH"])

tmp_g = G["8194nk5LbLH"]

pc = calculate_pc(path_data["path"], path, tmp_g)
print(pc)

cls = pc * calculate_ls(path_data, path, pc, tmp_g)

print(cls)

## Average Success Rate for test dataset (val_unseen), inference

In [ ]:
model_path1 = "./model_paths/dataset_v5_val_unseen_inst_0.json"
model_path2 = "./model_paths/dataset_v5_val_unseen_inst_1.json"
model_path3 = "./model_paths/dataset_v5_val_unseen_inst_2.json"

with open("tasks/R2R/data/R2R_val_unseen.json", "r", encoding="utf-8") as file:
    r2r_data = json.load(file)
    
print(f"Length of val unseen: {len(r2r_data)}")

model_path1_sr = get_success_rate(r2r_data, model_path1)
model_path2_sr = get_success_rate(r2r_data, model_path2)
model_path3_sr = get_success_rate(r2r_data, model_path3)

paths = set(model_path1_sr[-1] + model_path2_sr[-1]  + model_path3_sr[-1])
avg_sucess_rate = (model_path1_sr[0]+model_path2_sr[0]+model_path3_sr[0])/3
avg_navigation_error = (model_path1_sr[1]+model_path2_sr[1]+model_path3_sr[1])/3
print(f"Average Navigation Error: {avg_navigation_error}")
print(f"Average Success Rate for validation unseen data: {round(avg_sucess_rate, 2)}")

In [ ]:
path_mapping = get_path_mapping(r2r_data)
total_scans = set(p["scan"] for p in path_mapping.values())
print(total_scans)
scans = {path_mapping[int(p)]["scan"] : 0 for p in paths}
print(len(total_scans))

for p in paths:
    scans[path_mapping[int(p)]["scan"]] = scans.get(path_mapping[int(p)]["scan"], 0) + 1

print(scans)


In [ ]:
model_path1 = "./model_paths/qwen2_5_val_seen_test_instruction_1.json"
#model_path2 = "./model_paths/dataset_v5_val_seen_inst_1.json"
#model_path3 = "./model_paths/dataset_v5_val_seen_inst_2.json"

with open("tasks/R2R/data/R2R_val_seen.json", "r", encoding="utf-8") as file:
    r2r_data = json.load(file)
    
print(f"Length of val seen: {len(r2r_data)}")
    
model_path1_sr = get_success_rate(r2r_data, model_path1)
#model_path2_sr = get_success_rate(r2r_data, model_path2)[0]
#model_path3_sr = get_success_rate(r2r_data, model_path3)[0]
print(model_path1_sr[0])
print(model_path1_sr[1])
#avg_sucess_rate = (model_path1_sr+model_path2_sr+model_path3_sr)/3
#print(f"Average Success Rate for validation seen data: {round(avg_sucess_rate, 2)}")

In [ ]:
with open("./model_paths/qwen2_5_val_unseen_inst_0.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    
    
with open("tasks/R2R/data/R2R_val_unseen.json", "r", encoding="utf-8") as file:
    r2r_data = json.load(file)

print(len(data))
print(len(r2r_data))
r2r_mapping = get_path_mapping(r2r_data)
scans = set([p["scan"] for p in r2r_mapping.values()])

G = load_nav_graphs(scans)

In [ ]:
sr = 0
success_paths = []
longest_path = 0
longest_path_id = ""
for path_id, path in data.items():
    path_data = r2r_mapping[int(path_id)]
    gold_node = path_data["path"][-1]
    tmp_g = G[path_data["scan"]]
    neighbors = list(tmp_g.neighbors(gold_node))
    neighbors = [view for view, w in tmp_g[gold_node].items() if w["weight"] < 3]
    neighbors.append(gold_node)

    if path[-1] in neighbors:
        success_paths.append(path_id)
        sr += 1
        
        if len(path) > longest_path:
            longest_path = len(path)
            longest_path_id = path_id
        
print(f"success rate within 3 meters: {sr/len(data)}")        

In [ ]:
print(success_paths)

In [ ]:
print(longest_path)
print(longest_path_id)